**IMPORT**

In [1]:
import os
import pandas as pd
import tensorflow as tf

In [2]:
def normalize_series(data, min, max):
    data-=min
    data/=max
    return data

def windowed_dataset(series, batch_size, n_past=24, n_future=4, shift=1):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(size=n_past+n_future, shift=shift, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(n_past+n_future))
    ds = ds.shuffle(1000)
    ds = ds.map(lambda w: (w[:-n_future], w[-n_future:, :1]))
    ds = ds.batch(batch_size).prefetch(shift)
    return ds

In [7]:
# Retrieve dataset
df = pd.read_csv(os.path.join('lettuce_growth_days', 'lettuce_dataset.csv'), sep=',', encoding='ISO-8859-1')

# Delete column date, plant_id
df = df.drop(columns=['Plant_ID', 'Date'])

# Number of features in dataset
N_FEATURES = df.shape[1]

print(df.head())
print(F'n features: {N_FEATURES}')

# Normalize data
data = df.values
split_time = int(len(data)*0.8)
test = int(len(data)*0.9)
data = normalize_series(data, data.min(axis=0), data.max(axis=0))

x_train = data[:split_time]
print(f'Train: {len(x_train)}')
x_valid = data[split_time:test]
print(f'Valid: {len(x_valid)}')
x_test = data[test:]
print(f'Test: {len(x_test)}')

# DO NOT CHANGE THIS
BATCH_SIZE = 32
N_PAST = 24  # Number of past time steps based on which future observations should be predicted
N_FUTURE = 24  # Number of future time steps which are to be predicted.
SHIFT = 1  # By how many positions the window slides to create a new window of observations.

# Code to create windowed train and validation datasets.
# Complete the code in windowed_dataset.
# YOUR CODE HERE
train_set = windowed_dataset(series=x_train, batch_size=BATCH_SIZE, n_past=N_PAST, n_future=N_FUTURE, shift=SHIFT)
# YOUR CODE HERE
valid_set = windowed_dataset(series=x_valid, batch_size=BATCH_SIZE, n_past=N_PAST, n_future=N_FUTURE, shift=SHIFT)

model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(64, 'relu', return_sequences=True, input_shape=[N_PAST, N_FEATURES]),
    # tf.keras.layers.Dense(64, input_shape=(N_PAST, N_FEATURES)),
    tf.keras.layers.Dense(64),
    tf.keras.layers.Dense(32),
    
    tf.keras.layers.Dense(N_FUTURE, 'linear'),
])

class StopWhenReachDesireMAE(tf.keras.callbacks.Callback):
    def __init__(self, monitor='mae', monitor2='val_mae', target=0.14):
        super(StopWhenReachDesireMAE, self).__init__()
        self.monitor = monitor
        self.monitor2 = monitor2
        self.target = target

    def on_epoch_end(self, epoch, logs=None):
        current = logs.get(self.monitor)
        current2 = logs.get(self.monitor2)
        if current is not None and current2 is not None:
            if current < self.target and current2 < self.target:
                print(
                    f'\nEpoch {epoch + 1}: {self.monitor} and {self.monitor2} have reached {self.target}. Stopping training.')
                self.model.stop_training = True

stop_callback = StopWhenReachDesireMAE('mae', 'val_mae', 0.05)

# Code to train and compile the model
# YOUR CODE HERE
model.compile(
    loss='mae',
    optimizer='adam',
    metrics=['mae']
)

model.fit(
    train_set,
    epochs=60,
    validation_data=valid_set,
    callbacks=[stop_callback]
)

# Evaluate the model
loss, mae = model.evaluate(valid_set)
print(f'Mean Absolute Error on validation data: {mae}')

# Prepare test set for prediction
test_set = windowed_dataset(series=x_test, batch_size=BATCH_SIZE, n_past=N_PAST, n_future=N_FUTURE, shift=SHIFT)

# Make predictions
y_pred = model.predict(test_set)

# Flatten predictions for easier comparison
y_pred_flat = y_pred.reshape(-1)

# Extract actual values for comparison
# Note: This step assumes you have the actual future values in x_test for comparison
# Adjust based on your data's actual structure
x_test_flat = []
for window in windowed_dataset(x_test, BATCH_SIZE, N_PAST, N_FUTURE, SHIFT):
    x_test_flat.extend(window[1].numpy().flatten())

# Ensure the lengths match
min_length = min(len(x_test_flat), len(y_pred_flat))
x_test_flat = x_test_flat[:min_length]
y_pred_flat = y_pred_flat[:min_length]

# Optional: Convert predictions and actual values to a more readable format
results = pd.DataFrame({'Actual': x_test_flat, 'Predicted': y_pred_flat})
print(results.head())

   Temperature (°C)  Humidity (%)  TDS Value (ppm)  pH Level  Growth Days
0              33.4            53              582       6.4            1
1              33.5            53              451       6.1            2
2              33.4            59              678       6.4            3
3              33.4            68              420       6.4            4
4              33.4            74              637       6.5            5
n features: 5
Train: 2535
Valid: 317
Test: 317
Epoch 1/60
78/78 [==============================] - 5s 44ms/step - loss: 0.1182 - mae: 0.1182 - val_loss: 0.1287 - val_mae: 0.1287
Epoch 2/60
78/78 [==============================] - 3s 40ms/step - loss: 0.0537 - mae: 0.0537 - val_loss: 0.0829 - val_mae: 0.0829
Epoch 3/60
78/78 [==============================] - 3s 42ms/step - loss: 0.0483 - mae: 0.0483 - val_loss: 0.0816 - val_mae: 0.0816
Epoch 4/60
78/78 [==============================] - 3s 40ms/step - loss: 0.0446 - mae: 0.0446 - val_loss: 0.0813 - v

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

# Load the data
data = pd.read_csv(os.path.join('lettuce_growth_days', 'lettuce_dataset.csv'), sep=',', encoding='ISO-8859-1')

# Convert 'Date' to numerical values if necessary
data['Date'] = pd.to_datetime(data['Date']).map(pd.Timestamp.toordinal)

# Normalize the input features
features = ['Date', 'Temperature (°C)', 'TDS Value (ppm)', 'pH Level']
target = 'Growth Days'

X = data[features]
y = data[target]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

class CustomCallback(tf.keras.callbacks.Callback):
    def __init__(self, monitor='mae', monitor1='val_mae', target=0.1):
        super(CustomCallback, self).__init__()
        self.monitor = monitor
        self.monitor1 = monitor1
        self.target = target
        
    def on_epoch_end(self, epoch, logs=None):
        if logs[self.monitor] is not None and logs[self.monitor1] is not None:
            if logs[self.monitor] < self.target and logs[self.monitor1] < self.target:
                print(f'\n{self.monitor}: {logs[self.monitor]} < {self.target}')
                print(f'\n{self.monitor1}: {logs[self.monitor1]} < {self.target}')
                self.model.stop_training = True

# Build the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')  # Linear activation for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

cust_callbacks = CustomCallback(monitor='mae', target=0.045)
# Train the model
history = model.fit(X_train, y_train, epochs=1000, validation_split=0.2, batch_size=32, callbacks=[cust_callbacks])

# Evaluate the model
loss, mae = model.evaluate(X_test, y_test)
print(f'Mean Absolute Error on test data: {mae}')

# Make predictions
y_pred = model.predict(X_test)

# Optional: Convert predictions and actual values to a more readable format
results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred.flatten()})
print(results.head())

Epoch 1/1000
64/64 [==============================] - 1s 6ms/step - loss: 636.4443 - mae: 21.6893 - val_loss: 474.6724 - val_mae: 18.3719
Epoch 2/1000
64/64 [==============================] - 0s 4ms/step - loss: 314.4940 - mae: 14.7052 - val_loss: 105.8040 - val_mae: 8.8184
Epoch 3/1000
64/64 [==============================] - 0s 4ms/step - loss: 51.4686 - mae: 5.8576 - val_loss: 24.7878 - val_mae: 3.8606
Epoch 4/1000
64/64 [==============================] - 0s 4ms/step - loss: 17.2699 - mae: 3.2616 - val_loss: 13.3006 - val_mae: 2.8115
Epoch 5/1000
64/64 [==============================] - 0s 4ms/step - loss: 11.8217 - mae: 2.7334 - val_loss: 10.8087 - val_mae: 2.5708
Epoch 6/1000
64/64 [==============================] - 0s 4ms/step - loss: 9.7952 - mae: 2.4922 - val_loss: 9.0836 - val_mae: 2.3330
Epoch 7/1000
64/64 [==============================] - 0s 4ms/step - loss: 8.2399 - mae: 2.2754 - val_loss: 7.7043 - val_mae: 2.1351
Epoch 8/1000
64/64 [==============================] - 0s 4m

In [9]:
# Evaluate the model
loss, mae = model.evaluate(X_test, y_test)
print(f'Mean Absolute Error on test data: {mae}')

# Make predictions
y_pred = model.predict(X_test)

# Optional: Convert predictions and actual values to a more readable format
results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred.flatten()})
print(results.head())

20/20 [==============================] - 0s 2ms/step - loss: 0.0028 - mae: 0.0395
Mean Absolute Error on test data: 0.039545509964227676
20/20 [==============================] - 0s 1ms/step
      Actual  Predicted
254       25  25.016523
3162      40  40.060436
969       14  14.055223
940       30  29.979607
331       10   9.949435


In [12]:
print(y_test)

254     25
3162    40
969     14
940     30
331     10
        ..
2590     8
789     14
3008    21
1080    34
533     29
Name: Growth Days, Length: 634, dtype: int64
